In [ ]:
from flashml import plot_dist
import random
x1 = [random.randint(0, 5) for _ in range(10)]
x2 = [random.randint(0, 200) for _x in range(1000)]
x3 = [None, *x2, None]
x4= [random.random() for _ in range(1000)]
x5 = [None, *x4, None]
x6 = ["hi there"  , "jonanatha", "hi there", "hi there "]
x7 = [None, *x6]
x8 = {"daniel": 10, "james": 10}
plot_dist(x1)
plot_dist(x2)
plot_dist(x3)
plot_dist(x4)
plot_dist(x5)
plot_dist(x6)
plot_dist(x7)       
plot_dist(x8)  

In [ ]:
from transformers import AutoTokenizer
from flashml.inspect import inspect_tokenizer_window

tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-270m")
inspect_tokenizer_window(tokenizer)

In [ ]:
from flashml import host_mlflow


host_mlflow()

In [ ]:
from flashml import BatchIterator

data = range(3, 1000)

it = BatchIterator(data, batch_size=10, gradient_accumulation_steps=2, eval_ratio=0.2)
for i in it:
    print(i)

In [ ]:
from flashml import send_telegram_notification

# Test it
send_telegram_notification("Training Done!")

Message sent successfully!
